# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233799537975                   -0.50    7.0
  2   -7.249820036320       -1.80       -1.39    1.0
  3   -7.251210661203       -2.86       -1.95    4.0
  4   -7.251119351008   +   -4.04       -1.98    4.0
  5   -7.251314409367       -3.71       -2.47    2.0
  6   -7.251337638400       -4.63       -3.02    2.0
  7   -7.251338755863       -5.95       -3.88    1.0
  8   -7.251338782318       -7.58       -3.95    5.0
  9   -7.251338797714       -7.81       -4.47    3.0
 10   -7.251338798335       -9.21       -4.73    2.0
 11   -7.251338798695       -9.44       -5.64    3.0
 12   -7.251338798704      -11.05       -5.98    3.0
 13   -7.251338798704      -12.36       -6.44    2.0
 14   -7.251338798705      -12.91       -6.83    3.0
 15   -7.251338798705      -13.88       -7.12    2.0
 16   -7.251338798705      -14.27       -8.01    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04316362804067629
[ Info: Arnoldi iteration step 2: normres = 0.5420020188504124
[ Info: Arnoldi iteration step 3: normres = 0.730868186365601
[ Info: Arnoldi iteration step 4: normres = 0.5336289800243119
[ Info: Arnoldi iteration step 5: normres = 0.3930978042188106
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (8.73e-02, 7.72e-02, 2.61e-01, 2.69e-01, 2.46e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3720698239919428
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.85e-02, 1.62e-01, 3.08e-01, 8.37e-02, 7.75e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0789589939594089
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.12e-03, 1.61e-02, 1.66e-02, 2.65e-02, 6.38e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10660446621604718
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.27e-05,